# SiPhON Phase 0.2: Yield Analysis

Monte Carlo yield analysis connecting fabrication process variation to thermal power budget.

## Objectives
1. Visualize process variation sampling
2. Compute heater power distribution via the sensitivity chain
3. Calculate yield metric (% devices within power budget)
4. Produce yield vs. tolerance curves
5. Quantify the "cost of variance"

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'python'))

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from siphon.ring import RingResonator, RingGeometry
from siphon.thermal import ThermalModel
from siphon.sensitivity import EffectiveIndexSolver, WaveguideGeometry
from siphon.variability import (
    YieldAnalyzer, FabricationConfig, MonteCarloConfig, quick_yield,
)

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

## 1. Setup: Build the Analysis Pipeline

In [ ]:
# Compute sensitivity
wg = WaveguideGeometry(width=500e-9, height=220e-9)
solver = EffectiveIndexSolver(wg)
n_g = 4.2
sens = solver.sensitivity(n_g=n_g)

# Build ring + thermal
geom = RingGeometry(radius=10e-6, kappa=0.2, alpha=2.0, n_eff=sens.n_eff, n_g=n_g)
ring = RingResonator(geom)
thermal = ThermalModel(ring)

# Define fabrication tolerances
fab = FabricationConfig(sigma_w=10e-9, sigma_h=5e-9)
mc = MonteCarloConfig(n_samples=10_000, seed=42, max_heater_power=10e-3)

# Run Monte Carlo
analyzer = YieldAnalyzer(ring, thermal, sens, fab, mc)
result = analyzer.run()

print(f"Ring: R=10um, n_eff={sens.n_eff:.4f}, n_g={n_g}")
print(f"Tolerances: sigma_w={fab.sigma_w*1e9:.0f}nm, sigma_h={fab.sigma_h*1e9:.0f}nm")
print(f"\nYield: {result.yield_percent:.1f}% (at {mc.max_heater_power*1e3:.0f}mW budget)")
print(f"Mean power: {result.mean_heater_power*1e3:.2f} mW")
print(f"P95: {result.p95_heater_power*1e3:.2f} mW")

## 2. Process Variation Samples

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))

# Scatter plot of sampled geometries
w_nm = result.width_samples * 1e9
h_nm = result.height_samples * 1e9

scatter = ax.scatter(w_nm, h_nm, c=result.heater_powers * 1e3,
                     s=3, alpha=0.5, cmap='RdYlGn_r', vmin=0,
                     vmax=mc.max_heater_power * 1e3 * 2)
plt.colorbar(scatter, ax=ax, label='Required Heater Power (mW)')

# Mark nominal
ax.plot(500, 220, 'k*', markersize=15, label='Nominal')

# Draw 1-sigma and 3-sigma ellipses
for nsig, ls in [(1, '-'), (3, '--')]:
    e = Ellipse((500, 220), width=2*nsig*fab.sigma_w*1e9,
                height=2*nsig*fab.sigma_h*1e9,
                facecolor='none', edgecolor='black', linestyle=ls, lw=1.5,
                label=f'{nsig}-sigma')
    ax.add_patch(e)

ax.set_xlabel('Waveguide Width (nm)')
ax.set_ylabel('Silicon Thickness (nm)')
ax.set_title('Monte Carlo Geometry Samples (colored by heater power)')
ax.legend(loc='upper left')
ax.set_aspect('equal')
plt.tight_layout()
plt.show()

## 3. Heater Power Distribution

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of heater powers
powers_mw = result.heater_powers * 1e3
ax1.hist(powers_mw, bins=60, density=True, color='steelblue', alpha=0.7,
         edgecolor='white', linewidth=0.5)
ax1.axvline(mc.max_heater_power * 1e3, color='red', lw=2, linestyle='--',
            label=f'P_max = {mc.max_heater_power*1e3:.0f} mW')
ax1.axvline(result.mean_heater_power * 1e3, color='orange', lw=2,
            label=f'Mean = {result.mean_heater_power*1e3:.2f} mW')
ax1.axvline(result.p95_heater_power * 1e3, color='green', lw=2, linestyle=':',
            label=f'P95 = {result.p95_heater_power*1e3:.2f} mW')

ax1.set_xlabel('Required Heater Power (mW)')
ax1.set_ylabel('Probability Density')
ax1.set_title(f'Heater Power Distribution (Yield = {result.yield_percent:.1f}%)')
ax1.legend()

# Histogram of wavelength shifts
shifts_nm = result.wavelength_shifts * 1e9
ax2.hist(shifts_nm, bins=60, density=True, color='coral', alpha=0.7,
         edgecolor='white', linewidth=0.5)
ax2.axvline(0, color='black', lw=1, linestyle='-')
ax2.set_xlabel('Resonance Wavelength Shift (nm)')
ax2.set_ylabel('Probability Density')
ax2.set_title('Wavelength Shift Distribution')

plt.tight_layout()
plt.show()

## 4. Yield vs. Tolerance Curves

In [ ]:
# Sweep sigma_w at different power budgets
sigma_range = np.linspace(2e-9, 25e-9, 20)
power_budgets = [5e-3, 10e-3, 20e-3]
colors = ['red', 'blue', 'green']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for pmax, color in zip(power_budgets, colors):
    mc_sweep = MonteCarloConfig(n_samples=10_000, seed=42, max_heater_power=pmax)
    analyzer_sweep = YieldAnalyzer(ring, thermal, sens, fab, mc_sweep)
    sigmas, yields, mean_powers = analyzer_sweep.sweep_tolerance(sigma_w_range=sigma_range)

    ax1.plot(sigmas * 1e9, yields * 100, color=color, lw=2,
             label=f'P_max = {pmax*1e3:.0f} mW')
    ax2.plot(sigmas * 1e9, mean_powers * 1e3, color=color, lw=2,
             label=f'P_max = {pmax*1e3:.0f} mW')

ax1.set_xlabel('Width Tolerance sigma_w (nm)')
ax1.set_ylabel('Yield (%)')
ax1.set_title('Yield vs. Fabrication Tolerance')
ax1.set_ylim(0, 105)
ax1.legend()

ax2.set_xlabel('Width Tolerance sigma_w (nm)')
ax2.set_ylabel('Mean Heater Power (mW)')
ax2.set_title('Mean Heater Power vs. Tolerance')
ax2.legend()

plt.tight_layout()
plt.show()

## 5. Cost of Variance Analysis

Quantify the benefit of improving fabrication tolerance.

In [ ]:
print("=" * 70)
print("COST OF VARIANCE ANALYSIS")
print("=" * 70)
print(f"\nRing: R=10um, P_max=10mW\n")

scenarios = [
    ("Aggressive", 3e-9, 1.5e-9),
    ("Tight",      5e-9, 2.5e-9),
    ("Standard",  10e-9, 5e-9),
    ("Relaxed",   15e-9, 7.5e-9),
    ("Loose",     20e-9, 10e-9),
]

header = f"{'Control Level':<14} {'sigma_w':<10} {'sigma_h':<10} {'Yield':<10} {'Mean P':<12} {'P95':<10}"
print(header)
print("-" * 70)

results_table = []
for label, sw, sh in scenarios:
    r = quick_yield(
        n_eff=sens.n_eff, n_g=n_g,
        sigma_w=sw, sigma_h=sh,
        max_power=10e-3, n_samples=10_000, seed=42,
    )
    results_table.append((label, sw, sh, r))
    print(f"{label:<14} {sw*1e9:<10.0f} {sh*1e9:<10.1f} "
          f"{r.yield_percent:<10.1f} {r.mean_heater_power*1e3:<12.2f} "
          f"{r.p95_heater_power*1e3:<10.2f}")

# Delta analysis
if len(results_table) >= 3:
    std_result = results_table[2][3]  # Standard
    tight_result = results_table[1][3]  # Tight
    delta_yield = tight_result.yield_percent - std_result.yield_percent
    delta_power = std_result.mean_heater_power - tight_result.mean_heater_power
    print(f"\n--- Impact of improving from Standard to Tight control ---")
    print(f"  Yield improvement: +{delta_yield:.1f} percentage points")
    print(f"  Mean power saving: {delta_power*1e3:.2f} mW")

## 6. Yield vs. Power Budget

In [ ]:
# Sweep power budget for different tolerance levels
power_range = np.linspace(1e-3, 30e-3, 30)
tol_scenarios = [
    ("Tight (5nm)", 5e-9, 2.5e-9),
    ("Standard (10nm)", 10e-9, 5e-9),
    ("Loose (15nm)", 15e-9, 7.5e-9),
]

fig, ax = plt.subplots(figsize=(10, 6))

for label, sw, sh in tol_scenarios:
    yields_p = []
    for pmax in power_range:
        r = quick_yield(
            n_eff=sens.n_eff, n_g=n_g,
            sigma_w=sw, sigma_h=sh,
            max_power=pmax, n_samples=5_000, seed=42,
        )
        yields_p.append(r.yield_percent)
    ax.plot(power_range * 1e3, yields_p, lw=2, label=label)

ax.set_xlabel('Maximum Heater Power Budget (mW)')
ax.set_ylabel('Yield (%)')
ax.set_title('Yield vs. Heater Power Budget')
ax.set_ylim(0, 105)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Summary

### Phase 0.2 Results

1. **Sensitivity model** (EIM): Maps geometry variation to n_eff and resonance wavelength
2. **Monte Carlo engine**: 10,000 samples in <1 second (vectorized NumPy)
3. **Yield metric**: Fraction of devices tunable within heater power budget
4. **Cost of variance**: Tighter fabrication control directly reduces power consumption and improves yield

### Key Insight

The thermal overhead yield metric captures what passive alignment cannot:
even if a device is "close" to the target wavelength, it may require
unacceptable heater power to tune precisely. This analysis quantifies
the tradeoff between fabrication control and thermal budget.

### Next Steps (Phase 0.3)

- Replace EIM with C++ 2D FDE solver for higher-accuracy sensitivities
- Compare analytical vs. numerical yield predictions
- Quantify the EIM approximation error on yield

In [ ]:
# Final summary table
print("\n" + "=" * 70)
print("PHASE 0.2 VARIABILITY & YIELD - SUMMARY")
print("=" * 70)
print(f"\nReference Device: R=10um, 500nm x 220nm Si wire")
print(f"Operating Wavelength: 1550 nm")
print(f"Fabrication: sigma_w={fab.sigma_w*1e9:.0f}nm, sigma_h={fab.sigma_h*1e9:.0f}nm")
print(f"Power Budget: {mc.max_heater_power*1e3:.0f} mW\n")

print(f"{'Metric':<35} {'Value':<20} {'Unit'}")
print("-" * 70)
print(f"{'n_eff (EIM)':<35} {sens.n_eff:<20.6f} -")
print(f"{'dn_eff/dw':<35} {sens.dn_eff_dw*1e-9:<20.4e} per nm")
print(f"{'dn_eff/dh':<35} {sens.dn_eff_dh*1e-9:<20.4e} per nm")
print("-" * 70)
print(f"{'MC Samples':<35} {result.n_samples:<20d} -")
print(f"{'Yield':<35} {result.yield_percent:<20.1f} %")
print(f"{'Mean Heater Power':<35} {result.mean_heater_power*1e3:<20.2f} mW")
print(f"{'Std Heater Power':<35} {result.std_heater_power*1e3:<20.2f} mW")
print(f"{'95th Percentile Power':<35} {result.p95_heater_power*1e3:<20.2f} mW")
print("=" * 70)